In [6]:
!pip install pulp
import pulp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

#to display all rows columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", -1)

In [7]:
df = pd.read_excel('Fantasy INDvsWI.xlsx')

In [8]:
df

,Unnamed: 0,Player,Team,Role,Credits,Points
0,1,Rohit Sharma,IND,Batsman,8.5,88
1,2,Yashasvi Jaiswal,IND,Batsman,5.5,90
2,3,Shubman Gill,IND,Batsman,8.0,20
3,4,Virat Kohli,IND,Batsman,8.5,80
4,5,Ajinkya Rahane,IND,Batsman,8.0,15
5,6,Ravindra Jadeja,IND,All-rounder,9.0,70
6,7,Shardul Thakur,IND,All-rounder,7.5,40
7,8,Ishan Kishan,IND,Wicket keeper,5.5,30
8,9,Ravichandran Ashwin,IND,Bowler,9.0,90
9,10,Jaydev Unadkat,IND,Bowler,6.0,20


In [9]:
df = df.join(pd.get_dummies(df[['Role', 'Team']]))

In [10]:
df['playing_xi']=1

In [11]:
df

,Unnamed: 0,Player,Team,Role,Credits,Points,Role_All-rounder,Role_Batsman,Role_Bowler,Role_Wicket keeper,Team_IND,Team_WI,playing_xi
0,1,Rohit Sharma,IND,Batsman,8.5,88,0,1,0,0,1,0,1
1,2,Yashasvi Jaiswal,IND,Batsman,5.5,90,0,1,0,0,1,0,1
2,3,Shubman Gill,IND,Batsman,8.0,20,0,1,0,0,1,0,1
3,4,Virat Kohli,IND,Batsman,8.5,80,0,1,0,0,1,0,1
4,5,Ajinkya Rahane,IND,Batsman,8.0,15,0,1,0,0,1,0,1
5,6,Ravindra Jadeja,IND,All-rounder,9.0,70,1,0,0,0,1,0,1
6,7,Shardul Thakur,IND,All-rounder,7.5,40,1,0,0,0,1,0,1
7,8,Ishan Kishan,IND,Wicket keeper,5.5,30,0,0,0,1,1,0,1
8,9,Ravichandran Ashwin,IND,Bowler,9.0,90,0,0,1,0,1,0,1
9,10,Jaydev Unadkat,IND,Bowler,6.0,20,0,0,1,0,1,0,1


In [12]:
import pulp
import pandas as pd
df = pd.read_excel('Fantasy INDvsWI.xlsx')
df['playing_xi']=1
df['Team_IND'] = df['playing_xi'] * (df['Team'] == 'IND')
df['Team_WI'] = df['playing_xi'] * (df['Team'] == 'WI')
df['Role_Wicket keeper'] = (df['Role'] == 'Wicket keeper').astype(int)
df['Role_Batsman'] = (df['Role'] == 'Batsman').astype(int)
df['Role_Bowler'] = (df['Role'] == 'Bowler').astype(int)
df['Role_All-rounder'] = (df['Role'] == 'Role_All-rounder').astype(int)

#def decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary')

#def objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Points'] for obj in df.index])

#def constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Credits'] for obj in df.index]) <= 100

#11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'playing_xi'] for obj in df.index]) == 11

#team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Team_IND'] for obj in df.index]) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Team_IND'] for obj in df.index]) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Team_WI'] for obj in df.index]) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Team_WI'] for obj in df.index]) <= 6

#role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_Wicket keeper'] for obj in df.index]) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_Wicket keeper'] for obj in df.index]) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_Batsman'] for obj in df.index]) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_Batsman'] for obj in df.index]) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_Bowler'] for obj in df.index]) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_Bowler'] for obj in df.index]) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_All-rounder'] for obj in df.index]) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Role_All-rounder'] for obj in df.index]) <= 2

In [13]:
mo.solve()

-1

In [14]:
# output as a column in df
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

In [15]:
fdf = df[df['pickup_status'] == 1].sort_values(['Role_Batsman', 'Role_Wicket keeper', 'Role_All-rounder', 'Role_Bowler'], ascending = False).reset_index(drop = True)

In [16]:
fdf

,Unnamed: 0,Player,Team,Role,Credits,Points,playing_xi,Team_IND,Team_WI,Role_Wicket keeper,Role_Batsman,Role_Bowler,Role_All-rounder,pickup_status
0,1,Rohit Sharma,IND,Batsman,8.5,88,1,1,0,0,1,0,0,1
1,2,Yashasvi Jaiswal,IND,Batsman,5.5,90,1,1,0,0,1,0,0,1
2,4,Virat Kohli,IND,Batsman,8.5,80,1,1,0,0,1,0,0,1
3,16,Alick Athanaze,WI,Batsman,5.5,62,1,0,1,0,1,0,0,1
4,8,Ishan Kishan,IND,Wicket keeper,5.5,30,1,1,0,1,0,0,0,1
5,9,Ravichandran Ashwin,IND,Bowler,9.0,90,1,1,0,0,0,1,0,1
6,21,Kemar Roach,WI,Bowler,8.5,30,1,0,1,0,0,1,0,1
7,6,Ravindra Jadeja,IND,All-rounder,9.0,70,1,1,0,0,0,0,0,1
8,7,Shardul Thakur,IND,All-rounder,7.5,40,1,1,0,0,0,0,0,1
9,18,Jason holder,WI,All-rounder,8.5,35,1,0,1,0,0,0,0,1


In [17]:
sum(fdf.Credits)

83.5